In [1]:
"""try to use nbdev @jeremyhoward to write scripts"""
import numpy as np

In [2]:
class Tensor:
    def __init__(self,child=None,h=1,w=1,weight=None,trainable=True,grad=None):
        self.weight = weight
        self.trainable = trainable
        self.grad = grad
        self.child = child

a = Tensor()
b = Tensor(a)
a, b, b.child

(<__main__.Tensor at 0x2b71c77fac8>,
 <__main__.Tensor at 0x2b71c77fac8>)

In [3]:
class Act(Tensor):
    def __init__(self):
        super().__init__()
    
    def forward(self,x):
        raise NotImplementedError
    
    def backward(self,bpass):
        if self.child is not None:
            # backprop without assigning variable explicitly
            self.child.backward(np.multiply(self.grad,bpass))

In [4]:
class relu(Act):
    def __init__(self):
        super().__init__()
    
    def forward(self,x):
        out = np.maximum(x,0)
        self.grad = (out > 0).astype(np.float32)
        return out

x = np.arange(-1.,1.,0.5)
print("x: ",x)
xp = relu()
xp.forward(x), xp.grad

x:  [-1.  -0.5  0.   0.5]


(array([0. , 0. , 0. , 0.5]), array([0., 0., 0., 1.], dtype=float32))

In [5]:
class sigmoid(Act):
    def __init__(self):
        super().__init__()
    
    def forward(self,x):
        S = np.array(list(map(lambda x: 1/(1+np.exp(-x)), x)))
        self.grad = np.multiply(S, (1-S))
        return S

xpp = sigmoid()
xpp.forward(x), xpp.grad

(array([0.26894142, 0.37754067, 0.5       , 0.62245933]),
 array([0.19661193, 0.23500371, 0.25      , 0.23500371]))

In [6]:
class Layer(Tensor):
    def __init__(self,child=None):
        super().__init__()
        self.fpass = None
        self.child = child

    def forward(self,x):
        raise NotImplementedError
    
    def backward(self,grad):
        raise NotImplementedError

c = Layer()
d = Layer(c)
assert c == d.child
print(c.weight, c.trainable, c.grad)

None True None
